In [ ]:
from VR_Trajectory_analysis import *
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/df_cut.pkl'
results_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/color_bias_geometry3/results_df.pkl'
df_cut = pd.read_pickle(df_cut_path_pickle)
results_df = pd.read_pickle(results_df_path_pickle)

In [ ]:
df_joined = pd.merge(
    df_cut,
    results_df[['UniqueTrialID', 'FirstReachedGoal']],
    on='UniqueTrialID',
    how='left'
)
#add angle column to df_joined
df_joined['Angle'] = df_joined['ConfigFile'].apply(lambda x: parse_angle_from_config(x, default_angle=999.0))

In [ ]:
# -----------------------------------------------------------------
#  Trajectories for all  "BlueCylinder_BlueGreenCylinder" configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec

# 1.  The joined dataframe is assumed to exist:  df_joined

# 2.  Keep only the configurations we care about ---------------------------
accept_tags = ("BlueCylinder_BlueGreenCylinder",
               "Blue_BlueGreen")

df_filt = df_joined[df_joined['ConfigFile'].str.contains('|'.join(accept_tags))]

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*4, n_rows*3.75))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # trajectories
    for trial_id, td in g.groupby('UniqueTrialID'):
        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.1)

    # basic aesthetics
    ax.set_xlim(-40, 40)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)

    # left / right counts & ratios
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    tot   = left + right
    txt   = f"L: {left} ({left/tot:.2f})\nR: {right} ({right/tot:.2f})" if tot else "no goals"

    ax.text(.02, .98, txt, transform=ax.transAxes, va='top',
            fontsize=8, bbox=dict(boxstyle="round", fc="w", ec=".5"))

# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Trajectories – BlueCylinder_BlueGreenCylinder", y=.92, fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# ------------------------------------------------------------
# Preferred-goal reach vs platform angle   (all two-goal configs)
# ------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint   # pip install statsmodels

# ── 1.  Merge goal info into movement dataframe ──────────────────────────
# (uncomment if you haven’t created df_joined yet)
# df_joined = pd.merge(
#     df_cut,
#     results_df[['UniqueTrialID', 'FirstReachedGoal']],
#     on='UniqueTrialID',
#     how='left'
# )

# ensure Angle is numeric
df_joined['Angle'] = pd.to_numeric(df_joined['Angle'], errors='coerce')

# ── 2.  Per-config preferred-side counts ─────────────────────────────────
records = []

for cfg, g in df_joined.groupby('ConfigFile'):

    # skip single-goal layouts if you keep such a list
    if 'center_only_configs' in globals() and cfg in center_only_configs:
        continue

    # ---- goal counts for THIS config ------------------------------------
    res   = results_df.loc[results_df['UniqueTrialID'].isin(g['UniqueTrialID'])]
    left  = (res['FirstReachedGoal'] == 'left').sum()
    right = (res['FirstReachedGoal'] == 'right').sum()
    n     = left + right
    if n == 0:
        continue

    # ---- which platform is on the left / right? -------------------------
    if   "BlueGreenCylinder_BlueCylinder" in cfg or "BlueGreen_Blue" in cfg:
        label, colour, pref_cnt = 'BG→B', 'tab:blue',  left   # preferred = Left
    elif "BlueCylinder_BlueGreenCylinder" in cfg or "Blue_BlueGreen" in cfg:
        label, colour, pref_cnt = 'B→BG', 'tab:green', right  # preferred = Right
    else:
        label, colour, pref_cnt = 'other', 'tab:orange', left # symmetrical

    # ---- angle ----------------------------------------------------------
    angle_vals = g['Angle'].dropna().unique()
    if len(angle_vals) != 1:
        print(f'⚠️  Angle ambiguous → skipped: {cfg}')
        continue
    angle = float(angle_vals[0])

    records.append(dict(
        angle=angle, label=label, colour=colour,
        pref_cnt=pref_cnt, n_trials=n
    ))

# ── 3.  Aggregate over *all* configs that share the same angle ───────────
df  = pd.DataFrame(records)
agg = (df.groupby(['angle', 'label', 'colour'], as_index=False)
         .agg(pref_cnt=('pref_cnt', 'sum'),
              n_trials =('n_trials', 'sum')))

# Wilson CIs & preferred-goal proportion
lo, hi = [], []
for s, n in zip(agg['pref_cnt'], agg['n_trials']):
    l, h = proportion_confint(s, n, alpha=.05, method='wilson')
    lo.append(l); hi.append(h)

agg['p_pref']  = agg['pref_cnt'] / agg['n_trials']
agg['lo_pref'] = lo
agg['hi_pref'] = hi
# ------------------------------------------------------------------------

# ── 4.  Plot ─────────────────────────────────────────────────────────────
label_map = {'BG→B': 'Left', 'B→BG': 'Right', 'other': 'Symmetric'}

plt.figure(figsize=(6, 4))

for lbl, grp in agg.groupby('label'):
    plt.scatter(grp['angle'], grp['p_pref'],
                c=grp['colour'],
                label=label_map[lbl],
                s=60, alpha=.9)

    grp = grp.sort_values('angle')
    plt.plot(grp['angle'], grp['p_pref'],
             c=grp['colour'].iloc[0], lw=1, alpha=.4)

    plt.errorbar(grp['angle'], grp['p_pref'],
                 yerr=[grp['p_pref'] - grp['lo_pref'],
                       grp['hi_pref'] - grp['p_pref']],
                 fmt='none',
                 ecolor=grp['colour'].iloc[0],
                 alpha=.5, capsize=3)

plt.axhline(.5, ls='--', alpha=.4)
plt.xlabel('Platform angle (deg)')
plt.ylabel('Fraction of runs reaching preferred goal')
plt.title('Preferred-goal reach vs. platform angle   (95 % Wilson CIs)')
plt.legend(title='Preferred Goal')
plt.tight_layout()
plt.show()

In [ ]:
df_cut.groupby("ColonyID")["FlyID"].nunique()

There is one Ant without colony ID

In [ ]:
df_cut[df_cut["ColonyID"]== 'c']['FlyID'].unique()

In [ ]:
df_cut[df_cut["ColonyID"]== 'c']['Current Time']

In [ ]:
df_cut["ColonyID"] = df_cut["FlyID"].str[0]

In [ ]:
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ── 0) START: merge once to get the goal info if not already there ──────────────
if 'FirstReachedGoal' not in df_cut.columns:
    df_joined = (
        pd.merge(
            df_cut,                                  # has FlyID etc.
            results_df[['UniqueTrialID', 'FirstReachedGoal']],
            on='UniqueTrialID',
            how='left')
          .copy()
    )
else:
    df_joined = df_cut.copy()

# If the merge produced a suffixed name, normalise it
if 'FirstReachedGoal' not in df_joined.columns:
    cand = [c for c in df_joined.columns if c.startswith('FirstReachedGoal')]
    if not cand:
        raise KeyError("No 'FirstReachedGoal' column found after merge.")
    df_joined.rename(columns={cand[0]: 'FirstReachedGoal'}, inplace=True)

# ── 1) keep *two-goal* configurations only ─────────────────────────────────────
two_goal_mask = (
    df_joined['ConfigFile'].str.contains("BlueGreenCylinder_BlueCylinder") |
    df_joined['ConfigFile'].str.contains("BlueCylinder_BlueGreenCylinder")
)
df_pref = df_joined.loc[two_goal_mask].copy()

# ── 2) vectorised “preferred side” lookup ──────────────────────────────────────
pref_side = np.where(
    df_pref['ConfigFile'].str.contains("BlueGreenCylinder_BlueCylinder"),
    'left',          # BG→B
    'right'          # B→BG
)
df_pref['is_pref'] = (df_pref['FirstReachedGoal'] == pref_side)

# ── 3) per-fly fraction (drop flies with < 3 trials per config) ────────────────
fly_cfg = (
    df_pref
      .groupby(['ConfigFile', 'FlyID'])
      .agg(ratio_pref=('is_pref', 'mean'), n_trials=('is_pref', 'size'))
      .reset_index()
      .query('n_trials >= 3')
)

# ── 4) extract the numeric angle for the x-axis ────────────────────────────────
angle_pat = re.compile(r'(\d+)[°d]')
fly_cfg['angle'] = (
    fly_cfg['ConfigFile'].str.extract(angle_pat)[0].astype(int)
)

# ── 5) violin plot with Matplotlib (no seaborn needed) ─────────────────────────
plt.figure(figsize=(10, 5), dpi=120)

# build a list of datasets ordered by angle
angles = sorted(fly_cfg['angle'].unique())
data   = [fly_cfg.loc[fly_cfg['angle'] == a, 'ratio_pref'] for a in angles]

parts = plt.violinplot(data, positions=angles, widths=3,
                       showmeans=False, showmedians=False, showextrema=False)

# cosmetic tweaks
for pc in parts['bodies']:
    pc.set_facecolor('#5A9EBB')
    pc.set_edgecolor('k')
    pc.set_alpha(0.6)

# median + quartile markers
for xpos, vals in zip(angles, data):
    if len(vals) == 0:          # safety
        continue
    q1, med, q3 = np.percentile(vals, [25, 50, 75])
    plt.scatter([xpos]*3, [q1, med, q3], c='k', s=12, zorder=3)
    plt.plot([xpos, xpos], [q1, q3], c='k', lw=1)   # IQR bar

plt.axhline(0.5, ls='--', c='k', lw=.8, alpha=.5)
plt.xlabel('Platform angle (°)')
plt.ylabel('Fraction of that fly’s trials\nending in preferred goal')
plt.title('Per-fly preference ratios by platform angle')
plt.xlim(min(angles)-5, max(angles)+5)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate number of unique trials per FlyID
counts = df_cut.groupby('FlyID')['UniqueTrialID'].nunique()

# Plot histogram
plt.figure(figsize=(8, 4))
plt.hist(counts, bins='auto')  # default settings adhere to style rules
plt.xlabel('Number of unique trials per Fly')
plt.ylabel('Number of flies')
plt.title('Histogram of trial counts per FlyID')
plt.tight_layout()
plt.show()